## MLflow를 활용해 모델 서빙하기

### MLflow에 활용할  Info 만들기

In [1]:
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC

iris = load_iris()

data = pd.DataFrame(iris["data"], columns=iris["feature_names"])
target = pd.DataFrame(iris["target"], columns=["target"])

clf = SVC(kernel="rbf")
clf.fit(data, target)

input_example = data.sample(1)
signature = infer_signature(data, clf.predict(data))
conda_env = _mlflow_conda_env(additional_pip_deps=["dill", "pandas", "scikit-learn"])

/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 모델과 info 저장

In [2]:
from mlflow.sklearn import save_model

save_model(
    sk_model=clf,
    path="svc",
    serialization_format="cloudpickle",
    conda_env=conda_env,
    signature=signature,
    input_example=input_example,
)

### mlflow ui 생성
* bash에 `mlflow ui` 입력한 뒤 테이블에 있는 시간 누르면 해당 페이지로 이동

<br/>

<img src='img/mlrun.png' width='500px'>

In [3]:
import mlflow

with mlflow.start_run():
    mlflow.log_artifact("svc/")

### Mlflow component를 Kubeflow에서 사용하기